In [1]:
import argparse
import sys
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
class model():
    def __init__(self,profile="harmonic"):
        self.x = tf.placeholder(tf.float32, [None, 784])
        self.y = tf.placeholder(tf.int32,[None,10])
        self.idp = tf.placeholder(tf.float32,[None,1])
        self.learning_rate = 0.001
        
        self.W1 = tf.Variable(tf.random_uniform([784, 100],-0.1,0.1),name="W1")
        self.b1 = tf.Variable(tf.random_uniform([100],-0.1,0.1),name="b1")
        self.W2 = tf.Variable(tf.random_uniform([100, 100],-0.1,0.1),name="W2")
        self.b2 = tf.Variable(tf.random_uniform([100],-0.1,0.1),name="b2")
        self.W3 = tf.Variable(tf.random_uniform([100, 10],-0.1,0.1),name="W3")
        self.b3 = tf.Variable(tf.random_uniform([10],-0.1,0.1),name="b3")
        
        if profile == "linear":
            self.r1 = tf.constant_initializer(np.linspace(1,0,num=100,endpoint=False))
            self.r2 = tf.constant_initializer(np.linspace(1,0,num=100,endpoint=False))
            self.r3 = tf.constant_initializer(np.linspace(1,0,num=10,endpoint=False))
        if profile == "harmonic":
            self.r1 = tf.constant_initializer(1.0/(np.arange(100)+1))
            self.r2 = tf.constant_initializer(1.0/(np.arange(100)+1))
            self.r3 = tf.constant_initializer(1.0/(np.arange(10)+1))
        # t1
        n_ones = int(tf.size(self.r1) * idp)
        n_zeros = tf.size(self.r1) - ones
        self.t1 = tf.constant_initializer(np.hstack([np.ones(n_ones),np.zeros(n_zeros)]))
        
        # t2
        n_ones = int(tf.size(self.r2) * idp)
        n_zeros = tf.size(self.r2) - ones
        self.t2 = tf.constant_initializer(np.hstack([np.ones(n_ones),np.zeros(n_zeros)]))
        
        # t3
        n_ones = int(tf.size(self.r3) * idp)
        n_zeros = tf.size(self.r3) - ones
        self.t3 = tf.constant_initializer(np.hstack([np.ones(n_ones),np.zeros(n_zeros)]),dtype='float32')
                
        self.r1 = tf.multiply(self.r1,self.t1)
        self.r2 = tf.multiply(self.r2,self.t2)
        self.r3 = tf.multiply(self.r3,self.t3)
        
        rm_vars = [self.r1,self.r2,self.r3]
            
        # calculate r1*(W1*x)+b
        z1 = tf.add(tf.multiply(self.r1,tf.matmul(self.x,self.W1)),self.b1)
        y1 = tf.nn.relu(z1)
        z2 = tf.add(tf.multiply(self.r2,tf.matmul(self.y1,self.W2)),self.b2)
        y2 = tf.nn.relu(z2)
        logits = tf.add(tf.multiply(self.r3,tf.matmul(self.y2,self.W3)),self.b3)
        
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=self.y)
        
        self.loss = tf.reduce_sum(cross_entropy)
        
        tvars = tf.trainable_variables()
        
        tvars_rm = tf.train_variables()
        for rm in self.rm_vars:
            tvars_rm.remove(rm)
            
        self.train_all = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss,var_list=tvars)
        self.train_rm_gamma = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss,var_list=tvars_rm)

In [4]:
model = model()

TypeError: Failed to convert object of type <class 'tensorflow.python.ops.init_ops.Constant'> to Tensor. Contents: <tensorflow.python.ops.init_ops.Constant object at 0x7f1ee02c4748>. Consider casting elements to a supported type.

In [ ]:
def main(_):
    # Import data
    mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)
    
    # Create the model
    x = tf.placeholder(tf.float32, [None, 784])
    W1 = tf.Variable(tf.zeros([784, 10]))
    b1 = tf.Variable(tf.zeros([10]))
    W2 = tf.Variable(tf.)
    y = tf.matmul(x, W) + b

    # Define loss and optimizer
    y_ = tf.placeholder(tf.float32, [None, 10])
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
    
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    # Train
    for _ in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    # Test trained model
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels}))
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='/tmp/tensorflow/mnist/input_data',
                  help='Directory for storing input data')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)